## Introduction <a name="introduction"></a>

**A model to predict the estimated time of delivery of orders, from the point of driver pickup to the point of arrival at final destination. The solution will help Sendy enhance customer communication and improve the reliability of its service; which will ultimately improve customer experience. In addition, the solution will enable Sendy to realise cost savings, and ultimately reduce the cost of doing business, through improved resource management and planning for order scheduling.**

# Table of contents
1. [Introduction](#introduction)
2. [Importing libraries](#Importing_libraries)
3. [Importing datasets](#Importing_datasets)
4. [A quick look at how our data is structured](#Data_structure)
5. [Data Visualization](#visuals)
    1. [Temperature distribution](#temperature)
    2. [Vehicle types](#vehicle_types)
    3. [Platform types](#platform_types)
    4. [Personal or Business](#personal_or_business)
    5. [Order placement](#order_placement)
        1. [Order placement day of the month](#order_placement_day_of_the_month)
        2. [Placement weekday](#placement_weekday)
    6. [Order confirmation](#order_confirmation)
        1. [Confirmation day of month](#confirmation_day_of_month)
        2. [Confirmation weekday](#confirmation_weekday)
6. [Data Preprocessing](#data_preprocessing)
7. [Modeling](#modeling)
    1. [Linear Regression Model](#linear_model)
    2. [XGBoost](#xgb)
    3. [Random Forest](#random_forest)
    4. [Decision Tree](#decision_tree)
8. [Conclusion](#conclusion)